[View in Colaboratory](https://colab.research.google.com/github/apuneet/KGPP/blob/master/stqa.ipynb)

In [0]:
from keras import callbacks
from keras.utils import plot_model
from keras.preprocessing.sequence import pad_sequences
from keras import backend as ke
import os
import sys
import h5py
from shutil import copyfile
import numpy as np
import json
import pickle

lfn = None
cos_sim, tf_sess, a, b = None, None, None, None
is_theano = False
job_name = 'job-1'
load_legend = ['load from saved_data', 'save to saved_data', 'pre-process, but don\'t save or load']
p = dict()

In [0]:
def parse_params(conf_pf):
    global p
    sint = ignore_exception(ValueError)(int)
    sfloat = ignore_exception(ValueError)(float)
    with open(conf_pf) as f:
        content = f.readlines()
    for next_line in content:
        if next_line.startswith('#') or next_line == '\n' or len(next_line) == 0:
            continue
        toks = next_line.split('=')
        val = toks[1].replace('\n', '')
        print toks[0] + '=' + str(val)
        if sint(val) is not None:
            p[toks[0]] = sint(val)
            continue
        if sfloat(val) is not None:
            p[toks[0]] = sfloat(val)
            continue
        p[toks[0]] = val
    lod_home = 'data/Work-Homes/LOD_HOME/'
    p['dp_home'] = os.path.join(lod_home, p['dp_name'])
    my_args = MyArgs(p)
    jf, jn = set_job_folder(os.path.join(p['dp_home'], 'model/kgt1'))
    p['job_folder'] = jf
    p['job_number'] = jn
    p['conf_file'] = conf_pf
    my_args.set_details(jf, is_theano)
    copyfile(conf_pf, os.path.join(p['job_folder'], os.path.basename(conf_pf)))
    print '========================================================='
    return my_args


def set_job_folder(base_folder='/data/Work-Homes/LOD_HOME/fb15k2/model/kgt1/'):
    max_ct = 0
    for next_file in os.listdir(base_folder):
        if next_file.startswith('job-'):
            next_ct = int(next_file.replace('job-', ''))
            max_ct = max(max_ct, next_ct)
    job_number = int(max_ct+1)
    tb_log_folder = os.path.join(base_folder, 'job-' + str(job_number))
    print 'tb_log_folder=' + tb_log_folder
    os.mkdir(tb_log_folder)
    return tb_log_folder, job_number


class MyArgs:
    def __init__(self, ip):
        self.p = ip
        self.edge_list_pf = os.path.join(p['dp_home'], 'wip-data-3/converted-rdf-' + p['dp_name'] + '.rdf')
        self.job_folder = '/data/Work-Homes/LOD_HOME/fb15k2/model/kgt1/try/'
        self.word_embedding = os.path.join(ip['dp_home'], ip['embedding_suffix'])
        self.input_path = os.path.join(ip['dp_home'], ip['input_suffix'])
        self.is_theano = False
        #os.path.isdir(self.job_folder) or os.mkdir(self.job_folder)
        #self.input_path = '/data/Work-Homes/LOD_HOME/fb15k2/wip-data/query_prep/ready_question_sets/set-2'

    def set_details(self, jf, is_theano_used):
        self.job_folder = jf
        self.is_theano = is_theano_used


In [0]:
def main(conf_file):
    my_args = parse_params(conf_file)
    em_dim, word_ind, em_mat, isl, x_all, np_y_all, y_all, src_ent_all, osl, jos, qs, \
    rel_tok_seqs, rel_ids = load_saved_data(my_args)
    print_basics(em_dim, len(word_ind), len(em_mat), isl, osl)
    if p['load_command'] == 1:
        return
    x_train, y_train, x_valid, y_valid = x_all[0], np_y_all[0], x_all[1], np_y_all[1]
    print 'x_train.shape=' + str(x_train.shape)
    #print 'y_train.shape=' + str(y_train.shape)
    print 'isl=' + str(isl)
    store_settings(my_args)

In [26]:
main('kgt.conf')

dp_name=subgraph_webq
input_suffix=input_data/split-7
embedding_suffix=model/ere-glove/m-10.embd
use_ere=1
load_command=0
model_number=13
epoch_count=1
learn_embed=0
dropout=0.0
rec_dropout=0.0
l1=10
l2=6
l3=256
l4=256
pos_loss_wt=0.7
top_k=1
beam_size=1
none_val=0
tb_log_folder=data/Work-Homes/LOD_HOME/subgraph_webq/model/kgt1/job-4


IOError: ignored

In [0]:
def get_model13(p, em_mat, word_ind, em_dim, isl, osl):
    """
    Bi-LSTM, BatchNorm, MaxPool, 1-Layer Enc + BN, 1-Layer-Dec + BN + TDD-tanh
    """
    do = p['dropout']
    rdo = p['rec_dropout']
    fine_tune = True if p['learn_embed'] == 1 else False
    si = Input(shape=(isl,), dtype='int32')
    embedding_layer = Embedding(input_dim=len(word_ind) + 1, output_dim=em_dim,
                                input_length=isl, weights=[em_mat], trainable=fine_tune)(si)

    encoded = Bidirectional(LSTM(p['l1'], dropout=do, recurrent_dropout=rdo, return_sequences=True))(embedding_layer)
    bn_enc = normalization.BatchNormalization()(encoded)

    att = Dense(100, activation='softmax')(bn_enc)
    #pool_rnn = Lambda(lambda x: ke.max(x, axis=1))(bn_enc)
    #decode_inp = RepeatVector(osl)(att)
    #print 'em_dim=' + str(em_dim)

    decoded = Bidirectional(LSTM(p['l2'], dropout=do, recurrent_dropout=rdo, return_sequences=True))(att)
    td = TimeDistributed(Dense(em_dim, activation='tanh'))(decoded)

    s2s_model = Model(inputs=[si], outputs=[td])
    print 'Starting to compile the model ...'
    s2s_model.compile(optimizer='adam', loss=myloss)
    return s2s_model, True


def myloss(y_true, y_pred):
    v1 = y_pred
    v2 = y_true
    numerator = ke.sum(v1 * v2)
    denominator = ke.sqrt(ke.sum(v1 ** 2) * ke.sum(v2 ** 2))
    loss = abs(1 - numerator/denominator)
    return loss


In [0]:
def run_model(args, em_mat, word_ind, em_dim, isl, osl, x_train, y_train, x_valid, y_valid, x_test,
              rel_tok_seqs, rel_ids, y_all, src_ent_all, jos, qs):
    mods = get_model_dict()
    my_predictions_trn, my_predictions_tst = None, None
    
    my_mod, to_fp = get_model13(p, em_mat, word_ind, em_dim, isl, osl)
    my_predictions_trn, my_predictions_tst, emat, ygt = fit_and_predict(args, my_mod, x_train, y_train, x_valid, y_valid, x_test)    
    return my_predictions_trn, my_predictions_tst, emat, ygt


def fit_and_predict(my_args, my_model, x_train, y_train, x_valid, y_valid, x_test):
    plot_model(my_model, to_file=os.path.join(my_args.job_folder, 'model-' + str(p['job_number']) + '.png'))
    tb = callbacks.TensorBoard(log_dir=my_args.job_folder, histogram_freq=0, batch_size=32, write_graph=False,
                               write_grads=True, write_images=False, embeddings_freq=0,
                               embeddings_layer_names=None, embeddings_metadata=None)
    fp = my_args.job_folder + '/weights.{epoch:02d}-{val_loss:.4f}.hdf5'
    mc = callbacks.ModelCheckpoint(filepath=fp, save_best_only=True, mode='auto')
    print 'Starting to fit the model ...' + str(my_args.p['model_number'])
    if 'weight_file' in p.keys():
        fn = os.path.join(p['job_folder'], p['weight_file'])
        my_model.load_weights(fn)
    elif len(x_valid) < 5:
        if my_args.p['model_number'] != 12:
            my_model.fit(x=x_train, y=y_train, epochs=p['epoch_count'], verbose=1, callbacks=[tb])
        else:
            my_model.fit(x=x_train, y={'time_distributed_1': y_train[0], 'time_distributed_2': y_train[1]},
                         epochs=p['epoch_count'], verbose=1, callbacks=[tb])
    else:
        print 'Length of x_valid arrary = ' + str(len(x_valid))
        print 'y_train.shape=' + str(y_train.shape)
        print 'y_valid.shape=' + str(y_valid.shape)
        y_train1 = pad_sequences(y_train, maxlen=80, padding='post')
        y_valid1 = pad_sequences(y_valid, maxlen=80, padding='post')
        print 'y_train.shape=' + str(y_train1.shape)
        print 'y_valid.shape=' + str(y_valid1.shape)
        my_model.fit(x=x_train, y=y_train1, validation_data=(x_valid, y_valid1), epochs=p['epoch_count'], verbose=1,
                     callbacks=[tb, mc])

    print 'Starting to predict using the model ... with Tst Data First'
    my_predictions_tst = my_model.predict(x=x_test, verbose=1)
    print '\nmy_predictions_tst.shape=' + str(my_predictions_tst.shape)
    print 'my_predictions_tst[0].shape=' + str(my_predictions_tst[0].shape)
    print 'my_predictions_tst[1].shape=' + str(my_predictions_tst[1].shape)

    if my_predictions_tst.shape[1] > 2:
        y_temp = list()
        for i in range(len(my_predictions_tst)):
            mod_pred = my_predictions_tst[i][:2]
            y_temp.append(mod_pred)
        np_t_temp = np.asarray(y_temp, dtype=float)
        print 'new - my_predictions_tst.shape=' + str(np_t_temp.shape)
        my_predictions_tst = np_t_temp

    #my_predictions_trn = my_model.predict(x=x_train, verbose=1)
    my_predictions_trn = my_predictions_tst # temporarily disabling this
    if p['model_number'] == 12:
        tst_pres = my_predictions_tst[0]
        trn_pres = my_predictions_trn[0]
    else:
        tst_pres = my_predictions_tst
        trn_pres = my_predictions_trn
    return trn_pres, tst_pres, my_model.layers[1].get_weights()[0], None


def save_predictions(my_args, np_y_all_pred, split_name):
    print 'Entering save_predictions() - for ' + split_name
    print 'len(np_y_all_pred)=' + str(len(np_y_all_pred))
    print 'np_y_all_pred.shape=' + str(np_y_all_pred.shape)
    op_file = bas_utils.open_file(os.path.join(my_args.job_folder, split_name + '_pred_embeddings.txt'))
    for i in range(len(np_y_all_pred)):
        next_pred_list = np_y_all_pred[i]
        for pred_e in next_pred_list:
            op_file.write(bas_utils.to_string(pred_e, ',') + ';')
        op_file.write('\n')
    op_file.close()
    print 'Exiting save_predictions() - for ' + split_name


def myloss(y_true, y_pred):
    v1 = y_pred
    v2 = y_true
    numerator = ke.sum(v1 * v2)
    denominator = ke.sqrt(ke.sum(v1 ** 2) * ke.sum(v2 ** 2))
    loss = abs(1 - numerator/denominator)
    return loss


def print_basics(em_dim, len_word_ind, len_em_mat, isl, osl):
    print '--------------------------------------------------------------------------'
    print 'em_dim=' + str(em_dim)
    print 'len_word_ind=' + str(len_word_ind)
    print 'len_em_mat=' + str(len_em_mat)
    print 'Input Sequence Length=' + str(isl)
    print 'Output Sequence Length=' + str(osl)
    print '--------------------------------------------------------------------------'
    print '                              Loaded data '
    print '--------------------------------------------------------------------------'


def store_settings(args):
    op_file = open(os.path.join(args.job_folder, 'params.txt'), 'w')
    op_file.write('is_theano='+str(args.is_theano)+'\n')
    for k in p.keys():
        op_file.write(k+'=' + str(p[k]) + '\n')
    op_file.close()

In [0]:
def load_saved_data(this_args):
    op_path = os.path.abspath(os.path.join(this_args.job_folder, os.pardir, 'saved_data'))
    word_index = load_pickle_file(os.path.join(op_path, 'word_index.pickle'))
    embed_mat = load_pickle_file(os.path.join(op_path, 'embedding_matrix.pickle'))
    x_all_padded = load_pickle_file(os.path.join(op_path, 'x_all_padded.pickle'))
    np_y_all = load_pickle_file(os.path.join(op_path, 'np_y_all.pickle'))
    y_all = load_pickle_file(os.path.join(op_path, 'y_all.pickle'))
    src_ent_all = load_pickle_file(os.path.join(op_path, 'src_ent_all.pickle'))
    rel_tok_seqs = load_pickle_file(os.path.join(op_path, 'rel_tok_seqs.pickle'))
    rel_ids = load_pickle_file(os.path.join(op_path, 'rel_ids.pickle'))
    embd_dim, max_sen_len, max_osl = 0, 0, 0
    with open(os.path.join(op_path, 'vars.txt')) as f:
        content = f.readlines()
    for next_line in content:
        if next_line.startswith('embd_dim='):
            embd_dim = int(next_line.split('=')[1].replace('\n', ''))
        if next_line.startswith('max_sen_len='):
            max_sen_len = int(next_line.split('=')[1].replace('\n', ''))
        if next_line.startswith('max_osl='):
            max_osl = int(next_line.split('=')[1].replace('\n', ''))
    jos = list()
    for fn in fns:
        json_obj = bas_utils.load_json_file(os.path.join(op_path, fn))
        jos.append(json_obj)
    qs = load_pickle_file(os.path.join(op_path, 'qid_sqs.pickle'))
    print 'Returning from Saved Data ...'
    return embd_dim, word_index, embed_mat, max_sen_len, x_all_padded, np_y_all, y_all, src_ent_all, max_osl, jos, \
           qs, rel_tok_seqs, rel_ids


In [0]:
def ignore_exception(IgnoreException=Exception, DefaultVal=None):
    """ Decorator for ignoring exception from a function
    e.g.   @ignore_exception(DivideByZero)
    e.g.2. ignore_exception(DivideByZero)(Divide)(2/0)
    """
    def dec(function):
        def _dec(*args, **kwargs):
            try:
                return function(*args, **kwargs)
            except IgnoreException:
                return DefaultVal
        return _dec
    return dec

In [0]:
def load_pickle_file(path_file):
    return pickle.load(open(path_file, "rb"))


In [0]:
def load_json_file(path_file):
    print 'Starting to load JSON File - ' + path_file
    if not os.path.isfile(path_file):
        raise ValueError('No Such file - ' + path_file)
    ct, str_json,  = 0, ''
    with open(path_file) as f:
        content = f.readlines()
    for next_line in content:
        ct += 1
        str_json += next_line
        msg = 'load_json_file - ' + '     -->'
        print_status(msg, ct, 10)
    my_data_obj = json.loads(str_json)
    return my_data_obj